# CNN by PyTorch
Build the CNN model by PyTorch.<br>
#### Data
The data is CIFAR-10.<br>
Data Augmentation is used as follows:<br>
* Random horizontal flip
* Rotation: 0.2
#### Model
The model structure is as follows:<br>
* Convolutional layer 1: 32 filters, 3x3 kernel, stride 1, padding 1 + ReLU
* Convolutional layer 2: 32 filters, 3x3 kernel, stride 1, padding 1 + ReLU
* Max pooling layer 1: 2x2 kernel, stride 2
* Convolutional layer 3: 64 filters, 3x3 kernel, stride 1, padding 1 + ReLU
* Convolutional layer 4: 64 filters, 3x3 kernel, stride 1, padding 1 + ReLU
* Max pooling layer 2: 2x2 kernel, stride 2
* Flatten layer
* Fully connected layer 1: 256 units + ReLU + Dropout
* Fully connected layer 2: 10 units + Softmax
#### Training
* Optimizer: Adam
* Loss function: Cross Entropy
* Batch size: 32
* Epochs: 20
* Learning rate: 0.001
* early stopping: 5

## Load data

In [1]:
# Load the CIFAR10 dataset
import torch
from torchvision.datasets import CIFAR10

train_dataset = CIFAR10(root='data', train=True, download=True)
test_dataset = CIFAR10(root='data', train=False, download=True)

# Load the dataset into a DataLoader
from torch.utils.data import DataLoader

batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Data augmentation
from torchvision import transforms

train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10)
])

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting data/cifar-10-python.tar.gz to data
Files already downloaded and verified


## Build model

In [ ]:
# difine the model
# * Convolutional layer 1: 32 filters, 3x3 kernel, stride 1, padding 1 + ReLU
# * Convolutional layer 2: 32 filters, 3x3 kernel, stride 1, padding 1 + ReLU
# * Max pooling layer 1: 2x2 kernel, stride 2
# * Convolutional layer 3: 64 filters, 3x3 kernel, stride 1, padding 1 + ReLU
# * Convolutional layer 4: 64 filters, 3x3 kernel, stride 1, padding 1 + ReLU
# * Max pooling layer 2: 2x2 kernel, stride 2
# * Flatten layer
# * Fully connected layer 1: 256 units + ReLU + Dropout
# * Fully connected layer 2: 10 units + Softmax

import torch.nn as nn
import torch.nn.functional as F

input_size = 3 * 32 * 32

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(input_size, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv4 = nn.Conv2d(64, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 8 * 8, 256)
        self.fc2 = nn.Linear(256, 10)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = self.pool(x)
        x = self.dropout(F.relu(self.fc1(x.view(-1, 64 * 8 * 8))))
        x = F.softmax(self.fc2(x), dim=1)
        return x
    
model = CNN()
model.cuda()

## Train model

In [ ]:
# # train the model
# * Optimizer: Adam
# * Loss function: Cross Entropy
# * Batch size: 32
# * Epochs: 20
# * Learning rate: 0.001
# * early stopping: 5

# define the loss function and optimizer
import torch.optim as optim
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# log the loss and accuracy
recode_loss_train = []
recode_loss_test = []
recode_acc_train = []
recode_acc_test = []

# define the function to calculate the accuracy
def accuracy(y_pred, y_true):
    y_pred = torch.argmax(y_pred, dim=1)
    return torch.sum(y_pred == y_true).item() / len(y_true) * 100

# set the hyperparameters
epochs = 20
early_stopping = 5

# train the model
for epoch in range(epochs):
    # train the model
    model.train()
    loss_train = 0
    acc_train = 0
    for i, (x, y) in enumerate(train_loader):
        x = x.cuda()
        y = y.cuda()
        optimizer.zero_grad()
        y_pred = model(x)
        acc_train += accuracy(y_pred, y)
        loss = loss_fn(y_pred, y)
        loss_train += loss.item()
        loss.backward()
        optimizer.step()
    recode_loss_train.append(loss_train / len(train_loader))
    recode_acc_train.append(acc_train / len(train_loader))
    # test the model
    model.eval()
    loss_test = 0
    acc_test = 0
    for i, (x, y) in enumerate(test_loader):
        x = x.cuda()
        y = y.cuda()
        y_pred = model(x)
        acc_test += accuracy(y_pred, y)
        loss = loss_fn(y_pred, y)
        loss_test += loss.item()
    recode_loss_test.append(loss_test / len(test_loader))
    recode_acc_test.append(acc_test / len(test_loader))
    # print the loss and accuracy
    print('Epoch: {}/{}...'.format(epoch + 1, epochs),
          'Loss_train: {:.4f}...'.format(recode_loss_train[-1]),
          'Acc_train: {:.2f}...'.format(recode_acc_train[-1]),
          'Loss_test: {:.4f}...'.format(recode_loss_test[-1]),
          'Acc_test: {:.2f}...'.format(recode_acc_test[-1]))
    # early stopping
    if len(recode_loss_test) > early_stopping:
        if recode_loss_test[-1] > recode_loss_test[-1 - early_stopping]:
            print('Early stopping')
            break

## History

In [ ]:
# plot the loss and accuracy
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(recode_loss_train, label='train')
plt.plot(recode_loss_test, label='test')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.subplot(1, 2, 2)
plt.plot(recode_acc_train, label='train')
plt.plot(recode_acc_test, label='test')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()